<h1>
Model for Business Site Recommendation
</h1>

<ul>
<li>Build a dataframe of neighborhoods in Bangalore, India by web scraping the data from Wikipedia page</li>
<li>Get the geographical coordinates of the neighborhoods</li>
<li>Obtain the venue data for the neighborhoods from Foursquare API</li>
<li>Explore and cluster the neighborhoods</li>
<li>Select the best cluster to open a new shopping mall</li>
</ul>

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries imported.")


Libraries imported.


In [2]:
#Scrap data from Wikipedia page into a DataFrame
data = requests.get("https://en.m.wikipedia.org/wiki/List_of_neighbourhoods_in_Bangalore").text

In [3]:

# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')

In [4]:
neighborhoodList = []
for row in soup.find_all("div", class_="mw-parser-output")[0].findAll("a"):
    neighborhoodList.append(row.text)

In [5]:
b_df = pd.DataFrame({"Neighborhood": neighborhoodList[15:]})
b_df['Neighborhood'].replace('', inplace=True)
b_df.head(10)


,Neighborhood
0,Cantonment
1,British Raj
2,Richmond Town
3,Austin Town
4,Murphy Town
5,Fraser Town
6,Cox Town
7,Domlur
8,
9,HAL


In [6]:
b_df.shape


(187, 1)

In [7]:
# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Banaglore, India'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [8]:

# call the function to get the coordinates, store in a new list using list comprehension
coords = [ get_latlng(neighborhood) for neighborhood in b_df["Neighborhood"].tolist() ]

In [9]:
coords


[[12.994350000000054, 77.59839000000005],
 [12.966180000000065, 77.58690000000007],
 [12.962330000000065, 77.60123000000004],
 [12.963480000000061, 77.61297000000008],
 [12.980030000000056, 77.62776000000008],
 [12.998940000000061, 77.61276000000004],
 [12.995940000000076, 77.62644000000006],
 [12.943290000000047, 77.65602000000007],
 [12.966180000000065, 77.58690000000007],
 [12.960930000000076, 77.66419000000008],
 [13.030060000000049, 77.49526000000003],
 [12.966180000000065, 77.58690000000007],
 [12.908585406663308, 77.57026481360015],
 [13.005440000000021, 77.55693000000008],
 [13.00632005596653, 77.56839983128529],
 [12.966180000000065, 77.58690000000007],
 [12.966180000000065, 77.58690000000007],
 [12.966180000000065, 77.58690000000007],
 [12.975498768188668, 77.52385771447123],
 [12.970140000000072, 77.57589000000007],
 [12.95989017880979, 77.57431243116255],
 [12.969388412598315, 77.57950705966006],
 [13.014830000000075, 77.57771000000008],
 [12.961200000000076, 77.51071000000

In [10]:
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [11]:
b_df['Latitude'] = df_coords['Latitude']
b_df['Longitude'] = df_coords['Longitude']


In [12]:
print(b_df.shape)
b_df

(187, 3)


,Neighborhood,Latitude,Longitude
0,Cantonment,12.994350,77.598390
1,British Raj,12.966180,77.586900
2,Richmond Town,12.962330,77.601230
3,Austin Town,12.963480,77.612970
4,Murphy Town,12.980030,77.627760
5,Fraser Town,12.998940,77.612760
6,Cox Town,12.995940,77.626440
7,Domlur,12.943290,77.656020
8,,12.966180,77.586900
9,HAL,12.960930,77.664190


In [13]:
b_df.to_csv("b_df.csv", index=False)

Create a map of Bangalore with neighborhoods superimposed on top

In [38]:
# get the coordinates of Bangalore
address = 'Bangalore,India'

geolocator = Nominatim(user_agent="http")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Bangalore, India {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Bangalore, India 12.9767936, 77.590082.


In [15]:

map_b = folium.Map(location=[latitude, longitude], zoom_start=15)

# add markers to map
for lat, lng, neighborhood in zip(b_df['Latitude'], b_df['Longitude'], b_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_b)  
    
map_b

In [16]:
map_b.save('map_b.html')

In [17]:
CLIENT_ID = 'NZ0DM2ODTFWSE31NHMSMVZCX4A5YZMF4WKKVGGZTNCXVRXEM' # your Foursquare ID
CLIENT_SECRET = 'FDAGFHSOHZ2YTIACFGF2XY0ULG5A0EBLJTJETK3OVSCEOYJV' # your Foursquare Secret
VERSION = '20210609' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: NZ0DM2ODTFWSE31NHMSMVZCX4A5YZMF4WKKVGGZTNCXVRXEM
CLIENT_SECRET:FDAGFHSOHZ2YTIACFGF2XY0ULG5A0EBLJTJETK3OVSCEOYJV


In [18]:

radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(b_df['Latitude'], b_df['Longitude'],b_df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [19]:
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(12354, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Cantonment,12.99435,77.59839,Ujwal Bar & Restaurant,12.992280,77.594473,Indian Restaurant
1,Cantonment,12.99435,77.59839,Millers 46,12.991666,77.594207,Steakhouse
2,Cantonment,12.99435,77.59839,Infinitea,12.987157,77.594835,Tea Room
3,Cantonment,12.99435,77.59839,Alliance Française,12.991232,77.596723,Concert Hall
4,Cantonment,12.99435,77.59839,"Shangri-La Hotel, Bengaluru",12.992112,77.588446,Hotel


In [20]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
,700,700,700,700,700,700
100 Feet Road,98,98,98,98,98,98
Anekal,3,3,3,3,3,3
Anjanapura,7,7,7,7,7,7
Anugraha,22,22,22,22,22,22
Arekere,81,81,81,81,81,81
Attibele,8,8,8,8,8,8
Austin Town,100,100,100,100,100,100
Avenue Road,100,100,100,100,100,100


In [21]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 237 uniques categories.


In [22]:
venues_df['VenueCategory'].unique()[:50]


array(['Indian Restaurant', 'Steakhouse', 'Tea Room', 'Concert Hall',
       'Hotel', 'French Restaurant', 'Golf Course', 'Monument / Landmark',
       'Chinese Restaurant', 'Afghan Restaurant', 'Ice Cream Shop',
       'Cricket Ground', 'Park', 'Wine Shop', 'Art Museum', 'Pub',
       'Restaurant', 'Café', 'Burger Joint', 'Art Gallery', 'BBQ Joint',
       'Nightclub', 'Juice Bar', 'Lounge', 'Music Venue', 'Shopping Mall',
       "Women's Store", 'Hotel Bar', 'Fast Food Restaurant',
       'Pakistani Restaurant', 'Department Store', 'Pizza Place',
       'Snack Place', 'Bakery', 'Hyderabadi Restaurant',
       'Middle Eastern Restaurant', 'Vietnamese Restaurant',
       'Asian Restaurant', 'Clothing Store', 'Event Space', 'Pool Hall',
       'Coffee Shop', 'Bowling Alley', 'Sandwich Place', 'Candy Store',
       'Mediterranean Restaurant', 'Fried Chicken Joint', 'Food Truck',
       'Gym', 'Japanese Restaurant'], dtype=object)

In [23]:
"Neighborhood" in venues_df['VenueCategory'].unique()


True

In [24]:
b_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
b_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [b_onehot.columns[-1]] + list(b_onehot.columns[:-1])
b_onehot = b_onehot[fixed_columns]

print(b_onehot.shape)
b_onehot.head()

(12354, 238)


,Neighborhoods,ATM,Accessories Store,Afghan Restaurant,Airport Terminal,American Restaurant,Andhra Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Basketball Court,Bed & Breakfast,Beer Garden,Bengali Restaurant,Big Box Store,Bike Shop,Bistro,Boarding House,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Breakfast Spot,Brewery,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Campground,Candy Store,Capitol Building,Caribbean Restaurant,Castle,Chaat Place,Chettinad Restaurant,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cricket Ground,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Dive Bar,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Event Service,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Financial or Legal Service,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,General Entertainment,German Restaurant,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Health Food Store,Hill,Historic Site,History Museum,Hockey Arena,Hookah Bar,Hospital,Hotel,Hotel Bar,Hotel Pool,Hyderabadi Restaurant,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karnataka Restaurant,Kebab Restaurant,Kerala Restaurant,Korean Restaurant,Lake,Light Rail Station,Lighthouse,Liquor Store,Lounge,Maharashtrian Restaurant,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Motorcycle Shop,Movie Theater,Moving Target,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Music Store,Music Venue,Neighborhood,Nightclub,Noodle House,North Indian Restaurant,Office,Other Great Outdoors,Outlet Store,Paintball Field,Pakistani Restaurant,Paper / Office Supplies Store,Park,Parsi Restaurant,Performing Arts Venue,Pharmacy,Pizza Place,Planetarium,Platform,Playground,Plaza,Pool,Pool Hall,Pub,Punjabi Restaurant,Racecourse,Racetrack,Rajasthani Restaurant,Recreation Center,Residential Building (Apartment / Condo),Resort,Restaurant,Road,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,South Indian Restaurant,Southern / Soul Food Restaurant,Spa,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tea Room,Tech Startup,Tex-Mex Restaurant,Thai Restaurant,Theater,Tibetan Restaurant,Toll Booth,Tourist Information Center,Toy / Game Store,Track Stadium,Trail,Train Station,Travel & Transport,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Cantonment,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Cantonment,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [25]:
b_grouped = b_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(b_grouped.shape)
b_grouped

(127, 238)


,Neighborhoods,ATM,Accessories Store,Afghan Restaurant,Airport Terminal,American Restaurant,Andhra Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Basketball Court,Bed & Breakfast,Beer Garden,Bengali Restaurant,Big Box Store,Bike Shop,Bistro,Boarding House,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Breakfast Spot,Brewery,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Campground,Candy Store,Capitol Building,Caribbean Restaurant,Castle,Chaat Place,Chettinad Restaurant,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cricket Ground,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Dive Bar,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Event Service,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Financial or Legal Service,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,General Entertainment,German Restaurant,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Health Food Store,Hill,Historic Site,History Museum,Hockey Arena,Hookah Bar,Hospital,Hotel,Hotel Bar,Hotel Pool,Hyderabadi Restaurant,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karnataka Restaurant,Kebab Restaurant,Kerala Restaurant,Korean Restaurant,Lake,Light Rail Station,Lighthouse,Liquor Store,Lounge,Maharashtrian Restaurant,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Motorcycle Shop,Movie Theater,Moving Target,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Music Store,Music Venue,Neighborhood,Nightclub,Noodle House,North Indian Restaurant,Office,Other Great Outdoors,Outlet Store,Paintball Field,Pakistani Restaurant,Paper / Office Supplies Store,Park,Parsi Restaurant,Performing Arts Venue,Pharmacy,Pizza Place,Planetarium,Platform,Playground,Plaza,Pool,Pool Hall,Pub,Punjabi Restaurant,Racecourse,Racetrack,Rajasthani Restaurant,Recreation Center,Residential Building (Apartment / Condo),Resort,Restaurant,Road,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,South Indian Restaurant,Southern / Soul Food Restaurant,Spa,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tea Room,Tech Startup,Tex-Mex Restaurant,Thai Restaurant,Theater,Tibetan Restaurant,Toll Booth,Tourist Information Center,Toy / Game Store,Track Stadium,Trail,Train Station,Travel & Transport,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.010000,0.010000,0.000000,0.010000,0.010000,0.010000,0.000000,0.000000,0.000000,0.000000,0.010000,0.020000,0.000000,0.010000,0.000000,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.010000,0.000000,0.000000,0.040000,0.020000,0.010000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030000,0.000000,0.000000,0.010000,0.00,0.000,0.000000,0.000000,0.030000,0.000000,0.010000,0.010000,0.010000,0.00,0.000000,0.000000,0.000000,

In [26]:
len(b_grouped[b_grouped["Shopping Mall"] > 0])


55

In [27]:

b_mall = b_grouped[["Neighborhoods","Shopping Mall"]]
b_mall


,Neighborhoods,Shopping Mall
0,,0.010000
1,100 Feet Road,0.020408
2,Anekal,0.000000
3,Anjanapura,0.000000
4,Anugraha,0.000000
5,Arekere,0.024691
6,Attibele,0.000000
7,Austin Town,0.010000
8,Avenue Road,0.010000
9,BTM Layout,0.011111


Cluster Neighborhoods
Run k-means to cluster the neighborhoods in Bangalore into 3 clusters.

In [58]:
# set number of clusters
from sklearn.metrics import accuracy_score
bclusters = 3


b_clustering = b_mall.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=bclusters, random_state=0).fit(b_clustering)


# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]


C:\Users\nisch\AppData\Local\Temp\ipykernel_9124\2739157080.py:6: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  b_clustering = b_mall.drop(["Neighborhoods"], 1)


array([0, 2, 0, 0, 0, 2, 0, 0, 0, 0])

In [29]:
bmerged = b_mall.copy()

# add clustering labels
bmerged["Cluster Labels"] = kmeans.labels_

In [30]:

bmerged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
bmerged.head()

,Neighborhood,Shopping Mall,Cluster Labels
0,,0.010000,0
1,100 Feet Road,0.020408,2
2,Anekal,0.000000,0
3,Anjanapura,0.000000,0
4,Anugraha,0.000000,0


In [31]:
bmerged = bmerged.join(b_df.set_index("Neighborhood"), on="Neighborhood")

print(bmerged.shape)
bmerged.head() # check the last columns!

(187, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,,0.01,0,12.96618,77.5869
0,,0.01,0,12.96618,77.5869
0,,0.01,0,12.96618,77.5869
0,,0.01,0,12.96618,77.5869
0,,0.01,0,12.96618,77.5869


In [32]:
print(bmerged.shape)
bmerged.sort_values(["Cluster Labels"], inplace=True)
bmerged


(187, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,,0.010000,0,12.966180,77.586900
99,Rajarajeshwari Nagar,0.000000,0,12.931780,77.526680
100,Ramamurthy Nagar,0.000000,0,13.023810,77.677850
103,SH35,0.000000,0,12.905106,77.746767
103,SH35,0.000000,0,12.905106,77.746767
103,SH35,0.000000,0,12.905106,77.746767
104,SH39,0.000000,0,13.137835,77.479477
104,SH39,0.000000,0,13.137835,77.479477
105,SH87,0.010000,0,12.928380,77.600570
106,Sadashivanagar,0.000000,0,13.014830,77.577710


In [39]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(bclusters)
ys = [i+x+(i*x)**2 for i in range(bclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(bmerged['Latitude'], bmerged['Longitude'], bmerged['Neighborhood'], bmerged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [34]:
map_clusters.save('map_clusters.html')

In [35]:

bmerged.loc[bmerged['Cluster Labels'] == 0]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,,0.010000,0,12.966180,77.586900
99,Rajarajeshwari Nagar,0.000000,0,12.931780,77.526680
100,Ramamurthy Nagar,0.000000,0,13.023810,77.677850
103,SH35,0.000000,0,12.905106,77.746767
103,SH35,0.000000,0,12.905106,77.746767
103,SH35,0.000000,0,12.905106,77.746767
104,SH39,0.000000,0,13.137835,77.479477
104,SH39,0.000000,0,13.137835,77.479477
105,SH87,0.010000,0,12.928380,77.600570
106,Sadashivanagar,0.000000,0,13.014830,77.577710


In [55]:
bmerged.loc[bmerged['Cluster Labels'] == 1]


0.0


In [37]:
bmerged.loc[bmerged['Cluster Labels'] == 2]


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
66,Krishnarajapuram,0.032787,2,13.000390,77.683680
1,100 Feet Road,0.020408,2,12.908585,77.570265
5,Arekere,0.024691,2,12.885670,77.596730
68,Lingarajapuram,0.029851,2,13.005480,77.625970
74,Malleswaram,0.020000,2,13.006320,77.568400
73,Mahalakshmi Layout,0.017544,2,13.016350,77.544810
72,Mahadevapura,0.022222,2,12.994090,77.666330
60,Kammanahalli,0.015625,2,13.014260,77.636740
110,Thavarekere,0.020000,2,12.922400,77.610270
53,J. P. Nagar,0.020000,2,12.908310,77.590240
